# Inferring movie genre by its plot

In [178]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv 

import time

from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

%matplotlib inline
pd.set_option('display.max_colwidth', 300)

[nltk_data] Downloading package stopwords to /home/kelu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# The Dataset

* Using the `CMU Movie Summary Corpus` (check https://www.cs.cmu.edu/~ark/personas/data/README.txt )
* It contains movie names, genres, and plot: all we need to try and guess the genre from its plot!

# Diving in the data

In [5]:
meta = pd.read_csv("MovieSummaries/movie.metadata.tsv", sep = '\t', header = None)
meta.columns = ["movie_id",1,"movie_name",3,4,5,6,7,"genre"] #Let's discard the rest of the cols
meta.head()

,movie_id,1,movie_name,3,4,5,6,7,genre
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science Fiction"", ""/m/03npn"": ""Horror"", ""/m/03k9fj"": ""Adventure"", ""/m/0fdjb"": ""Supernatural"", ""/m/02kdv5l"": ""Action"", ""/m/09zvmj"": ""Space western""}"
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey Mystery,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0"": ""Drama"", ""/m/0hj3n01"": ""Crime Drama""}"
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""Drama""}"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic thriller"", ""/m/09blyk"": ""Psychological thriller""}"
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


# Creating the plots and genre columns

In [32]:
plots = []

with open("MovieSummaries/plot_summaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab') 
    for row in tqdm(reader):
        plots.append(row)
        
movie_id = []
plot = []

# extract movie Ids and plot summaries
for i in tqdm(plots):
    movie_id.append(i[0])
    plot.append(i[1])

# create dataframe
movies = pd.DataFrame({'movie_id': movie_id, 'plot': plot})

# change datatype of 'movie_id'
meta['movie_id'] = meta['movie_id'].astype(str)

# merge meta with movies
movies = pd.merge(movies, meta[['movie_id', 'movie_name', 'genre']], on = 'movie_id')

# an empty list
genres = [] 

# extract genres
for i in movies['genre']: 
    genres.append(list(json.loads(i).values())) 

# add to 'movies' dataframe  
movies['genres'] = genres


movies = movies[["movie_id","plot","movie_name","genres"]]
movies = movies[~(movies['genres'].str.len() == 0)]

movies

100%|██████████| 42303/42303 [00:00<00:00, 1080503.51it/s]


,movie_id,plot,movie_name,genres
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",Taxi Blues,"[Drama, World cinema]"
1,31186339,"The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl between the ages of 12 and 18 selected by lottery for the annual Hunger Games. The tributes must fight to the death in an arena; the sole...",The Hunger Games,"[Action/Adventure, Science Fiction, Action, Drama]"
2,20663735,"Poovalli Induchoodan is sentenced for six years prison life for murdering his classmate. Induchoodan, the only son of Justice Maranchery Karunakara Menon was framed in the case by Manapally Madhavan Nambiar and his crony DYSP Sankaranarayanan to take revenge on idealist judge Menon who had e...",Narasimham,"[Musical, Action, Drama, Bollywood]"
3,2231378,"The Lemon Drop Kid , a New York City swindler, is illegally touting horses at a Florida racetrack. After several successful hustles, the Kid comes across a beautiful, but gullible, woman intending to bet a lot of money. The Kid convinces her to switch her bet, employing a prefabricated con. Unfo...",The Lemon Drop Kid,"[Screwball comedy, Comedy]"
4,595909,"Seventh-day Adventist Church pastor Michael Chamberlain, his wife Lindy, their two sons, and their nine-week-old daughter Azaria are on a camping holiday in the Outback. With the baby sleeping in their tent, the family is enjoying a barbecue with their fellow campers when a cry is heard. Lindy r...",A Cry in the Dark,"[Crime Fiction, Drama, Docudrama, World cinema, Courtroom Drama]"
...,...,...,...,...
42199,34808485,"The story is about Reema , a young Muslim schoolgirl in Malabar, who loved Malayalam and poetry. The film is about her mental trauma once she was moved to an English medium school by her parents, for their social status.",Oomakkuyil Padumbol,[Children's]
42200,1096473,"In 1928 Hollywood, director Leo Andreyev looks through photographs for actors for his next movie. When he comes to the picture of an aged Sergius Alexander , he pauses, then tells his assistant to cast the man. Sergius shows up at the Eureka Studio with a horde of other extras and is issued a ...",The Last Command,"[Silent film, Indie, Black-and-white, Period piece, Drama, War film, Political drama]"
42201,35102018,"American Luthier focuses on Randy Parsons’ transformation from aspiring musician to guitar-maker. The guitar had been Parsons’ identity since he was a child, but after studying classical and jazz guitar in college, he realized that he would never make a living as a musician. So he gave it up — h...",Randy Parsons: American Luthier,"[Short Film, Music, Biographical film, Documentary]"
42202,8628195,"Abdur Rehman Khan , a middle-aged dry fruit seller from Afghanistan, comes to Calcutta to hawk his merchandise and befriends a small Bengali girl called Mini who reminds him of his own daughter Amina back in Afghanistan. He puts up at a boarding house along with his countrymen. Since he is shor...",Kabuliwala,[Drama]


# Data cleaning

In [37]:
# Removing stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

# All to lower
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return remove_stopwords(text)

In [38]:
movies['clean_plot'] = movies['plot'].apply(lambda x: clean_text(x))
movies

,movie_id,plot,movie_name,genres,clean_plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",Taxi Blues,"[Drama, World cinema]",shlykov hard working taxi driver lyosha saxophonist develop bizarre love hate relationship despite prejudices realize arent different
1,31186339,"The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl between the ages of 12 and 18 selected by lottery for the annual Hunger Games. The tributes must fight to the death in an arena; the sole...",The Hunger Games,"[Action/Adventure, Science Fiction, Action, Drama]",nation panem consists wealthy capitol twelve poorer districts punishment past rebellion district must provide boy girl ages selected lottery annual hunger games tributes must fight death arena sole survivor rewarded fame wealth first reaping year old primrose everdeen chosen district older siste...
2,20663735,"Poovalli Induchoodan is sentenced for six years prison life for murdering his classmate. Induchoodan, the only son of Justice Maranchery Karunakara Menon was framed in the case by Manapally Madhavan Nambiar and his crony DYSP Sankaranarayanan to take revenge on idealist judge Menon who had e...",Narasimham,"[Musical, Action, Drama, Bollywood]",poovalli induchoodan sentenced six years prison life murdering classmate induchoodan son justice maranchery karunakara menon framed case manapally madhavan nambiar crony dysp sankaranarayanan take revenge idealist judge menon earlier given jail sentence manapally corruption case induchoodan achi...
3,2231378,"The Lemon Drop Kid , a New York City swindler, is illegally touting horses at a Florida racetrack. After several successful hustles, the Kid comes across a beautiful, but gullible, woman intending to bet a lot of money. The Kid convinces her to switch her bet, employing a prefabricated con. Unfo...",The Lemon Drop Kid,"[Screwball comedy, Comedy]",lemon drop kid new york city swindler illegally touting horses florida racetrack several successful hustles kid comes across beautiful gullible woman intending bet lot money kid convinces switch bet employing prefabricated con unfortunately kid woman belongs notorious gangster moose moran money ...
4,595909,"Seventh-day Adventist Church pastor Michael Chamberlain, his wife Lindy, their two sons, and their nine-week-old daughter Azaria are on a camping holiday in the Outback. With the baby sleeping in their tent, the family is enjoying a barbecue with their fellow campers when a cry is heard. Lindy r...",A Cry in the Dark,"[Crime Fiction, Drama, Docudrama, World cinema, Courtroom Drama]",seventh day adventist church pastor michael chamberlain wife lindy two sons nine week old daughter azaria camping holiday outback baby sleeping tent family enjoying barbecue fellow campers cry heard lindy returns tent check azaria certain sees dingo something mouth running approaches discovers i...
...,...,...,...,...,...
42199,34808485,"The story is about Reema , a young Muslim schoolgirl in Malabar, who loved Malayalam and poetry. The film is about her mental trauma once she was moved to an English medium school by her parents, for their social status.",Oomakkuyil Padumbol,[Children's],story reema young muslim schoolgirl malabar loved malayalam poetry film mental trauma moved english medium school parents social status
42200,1096473,"In 1928 Hollywood, director Leo Andreyev looks through photographs for actors for his next movie. When he comes to the picture of an aged Sergius Alexander , he pauses, then tells his assistant to cast the man. Sergius shows up at the Eureka Studio with a horde of other extras and is issued a ...",The Last Command,"[Silent film, Indie, Black-and-white, Period piece, Drama, War film, Political drama]",hollywood director leo andreyev looks photographs 

In [159]:
#Let's save the dataframe for later as need be
movies.to_pickle("movies.pkl")

# Starting predictions

We'll be using several labels hence MultiLabelBinarizer

In [186]:
from sklearn.preprocessing import MultiLabelBinarizer

# Models
from sklearn.linear_model import Ridge,LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
# Perf
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score, balanced_accuracy_score
from sklearn.metrics import mean_squared_error

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(movies['genres'])

MultiLabelBinarizer()

# TFID preparation

Let's see if we can play with Number of features

In [172]:
NFeatures = 1000
Creg = 1.0

In [173]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=NFeatures)

# Split dataset into training and validation set

## Let's apply what we have:

Training, validation, and testing datasets are split there.

In [197]:
y = multilabel_binarizer.transform(movies['genres'])
xtrain, xtest, ytrain, ytest = train_test_split(movies['clean_plot'], y, test_size=0.2, random_state=9)
xtrain, xval, ytrain , yval  = train_test_split(xtrain, ytrain, test_size=0.25, random_state=1)
print("Train: ",len(xtrain)," - val: ",len(xval), " and test: ",len(xtest))
# create TF-IDF features
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)
xtest_tfidf = tfidf_vectorizer.transform(xtest)

Train:  25075  - val:  8359  and test:  8359


## Preparing the model

### Regularization is used as under the C parameter in `LogisticRegression`

The smaller C, the stronger the regularization.

### We also play with the `tfidf_vectorizer`, by playing with the number of features.

In [179]:
lr = LogisticRegression(C = Creg)
# Inverse of regularization strength; must be a positive float. Like in support vector machines, 
# smaller values specify stronger regularization.
ri = Ridge(alpha=0.1)
clf = OneVsRestClassifier(lr)

In [180]:
%%time
t0= time.clock()
clf.fit(xtrain_tfidf, ytrain)
TimeSpent = time.clock() - t0
# fit model on train data
TimeSpent

/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 48 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 168 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 182 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 214 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 245 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 268 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81:

CPU times: user 1min 7s, sys: 327 ms, total: 1min 7s
Wall time: 1min 8s


67.9057150000001

### Exploring diagnostics for the data

We try and use :
* from sklearn.metrics import __f1_score__
* from sklearn.metrics import __recall_score__
* from sklearn.metrics import __mean_squared_error__

In [181]:
# make predictions for validation set
y_pred = clf.predict(xval_tfidf)
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [182]:
y_pred_prob = clf.predict_proba(xval_tfidf)
y_pred_prob

array([[1.20626078e-03, 1.22717485e-04, 1.34763473e-01, ...,
        1.27486497e-03, 7.61983961e-05, 4.96766738e-03],
       [1.13298144e-03, 1.25919759e-04, 9.50193043e-03, ...,
        1.21796781e-03, 7.99191188e-05, 2.74872670e-03],
       [8.73526523e-04, 1.19307698e-04, 8.54691632e-02, ...,
        1.44666879e-03, 7.93343319e-05, 2.67407707e-03],
       ...,
       [1.49638814e-03, 1.14356004e-04, 7.20726063e-02, ...,
        3.23507520e-03, 8.80310935e-05, 5.19982265e-03],
       [1.28845537e-03, 1.20870202e-04, 2.46899730e-01, ...,
        2.10607184e-03, 8.36468423e-05, 7.24912178e-03],
       [1.58545246e-03, 1.34265946e-04, 7.10292686e-01, ...,
        4.62015922e-03, 8.44330874e-05, 5.25637926e-03]])

In [201]:
t = 0.40 # threshold value
y_pred_prob = clf.predict_proba(xval_tfidf)
y_pred_new = (y_pred_prob >= t).astype(int)

In [202]:
# evaluate performance
f1_score(yval, y_pred, average="micro"),f1_score(yval, y_pred_new, average="micro")

(0.2848950866675355, 0.35370106012028746)

In [203]:
recall_score(yval, y_pred, average="micro"),recall_score(yval, y_pred_new, average="micro")

(0.18240379159574113, 0.2522278962651447)

In [204]:
mean_squared_error(yval, y_pred),mean_squared_error(yval, y_pred_new)

(0.009041573441403782, 0.009101553990568553)

In [126]:
def infer_tags(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

# Now we can have fun and try and guess what movies are what =)

In [157]:
for i in range(5): 
    k = xval.sample(1).index[0] 
    print("Movie: ", movies['movie_name'][k], "\n -> Predicted genre: ", infer_tags(xval[k])), print(" -> Actual genre: ",movies['genres'][k], "\n")


Movie:  Sufferin' Cats 
 -> Predicted genre:  [('Animation', 'Comedy', 'Family Film', 'Short Film')]
 -> Actual genre:  ['Comedy film', 'Family Film', 'Animation'] 

Movie:  Løvejagten 
 -> Predicted genre:  [()]
 -> Actual genre:  ['Short Film', 'Silent film', 'Black-and-white'] 

Movie:  Jaii 
 -> Predicted genre:  [()]
 -> Actual genre:  ['Romance Film', 'Drama'] 

Movie:  Virginia's Run 
 -> Predicted genre:  [('Family Film',)]
 -> Actual genre:  ['Animal Picture', 'Indie', "Children's/Family", 'Family Drama', 'Coming of age', 'Drama', 'Family Film'] 

Movie:  The Rum Diary 
 -> Predicted genre:  [('Comedy', 'Drama', 'Romance Film')]
 -> Actual genre:  ['Thriller', 'Romance Film', 'Drama', 'Comedy', 'Adventure'] 



# Mapping the results and parameters

We try and explore fooling around with the number of features in the tfidf, as well as the regularization parameter in LinearRegression (_Creg_), and the threshold considered for our classifier.

We keep in mind the time spent to train the model as well, as a sort of a cost function.

In [207]:
Results = []

NFeatures = [250,1000,2500]
Creg = [0.1, 1.0, 10, 100]
t = [0.75,0.50,0.3]

for nFeature in NFeatures:
    for creg in Creg:
        print(nFeature,creg)
        
        tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=nFeature)

        lr = LogisticRegression(C = creg)
        clf = OneVsRestClassifier(lr)

        #%%time
        t0= time.clock()
        clf.fit(xtrain_tfidf, ytrain)
        TimeSpent = time.clock() - t0
        # fit model on train data

        # make predictions for validation set
        y_pred = clf.predict(xval_tfidf)
        
        for threshold in t:
                
             # threshold value
            y_pred_prob = clf.predict_proba(xval_tfidf)
            y_pred_new = (y_pred_prob >= threshold).astype(int)

            # evaluate performance
            F1_naive = f1_score(yval, y_pred, average="micro")
            F1_custm = f1_score(yval, y_pred_new, average="micro")

            REC_naive = recall_score(yval, y_pred, average="micro")
            REC_custm = recall_score(yval, y_pred_new, average="micro")

            MSE_naive = mean_squared_error(yval, y_pred)
            MSE_custm = mean_squared_error(yval, y_pred_new)
        
            Results.append([nFeature,creg,TimeSpent,threshold,MSE_naive,MSE_custm,F1_naive,F1_custm,REC_naive,REC_custm])



250 0.1


/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 48 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 168 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 182 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 214 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 245 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 268 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81:

250 1.0


/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 48 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 168 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 182 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 214 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 245 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 268 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81:

250 10


...

/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

/home/kelu/.local/lib/python3.6/site-packages/sklearn/multiclass.py:81: UserWarning: Label not 245 is present in all training examples.
  str(classes[c]))
/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/kelu/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

In [213]:
MyRes = pd.DataFrame.from_records(Results, columns=["nFeature","creg","TimeSpent","threshold","MSE_naive","MSE_custm","F1_naive","F1_custm","REC_naive","REC_custm"])
MyRes.to_pickle("results_models.pkl")
MyRes.sort_values(by="F1_custm",ascending=False).head(10)

,nFeature,creg,TimeSpent,threshold,MSE_naive,MSE_custm,F1_naive,F1_custm,REC_naive,REC_custm
8,250,10.0,114.678041,0.3,0.009224,0.010622,0.330639,0.408219,0.230733,0.371016
20,1000,10.0,113.324942,0.3,0.009224,0.010622,0.330639,0.408219,0.230733,0.371016
32,2500,10.0,125.481859,0.3,0.009224,0.010622,0.330639,0.408219,0.230733,0.371016
29,2500,1.0,58.659026,0.3,0.009042,0.009644,0.284895,0.403863,0.182404,0.330830
17,1000,1.0,59.402514,0.3,0.009042,0.009644,0.284895,0.403863,0.182404,0.330830
5,250,1.0,59.829008,0.3,0.009042,0.009644,0.284895,0.403863,0.182404,0.330830
23,1000,100.0,158.771310,0.3,0.010052,0.012118,0.329258,0.385069,0.249858,0.384266
11,250,100.0,162.445331,0.3,0.010052,0.012118,0.329258,0.385069,0.249858,0.384266
35,2500,100.0,170.854850,0.3,0.010052,0.012118,0.329258,0.385069,0.249858,0.384266
7,250,10.0,114.678041,0.5,0.009224,0.009224,0.330639,0.330639,0.230733,0.230733
